## Отчет по исследованию Looped Transformers и их возможностей.
Мини исследование-отчет по статье - https://arxiv.org/abs/2311.12424

### Сравнение эффективности Looped TFs с обычными TFs
Все эксперементы проводились с выключенным Mixed Precision, по скольку он не влиял на скорость обучения - видимо из-за того что gpt2_nano не может использовать GPU эффективно.
Одна часть эксперементов проводилась на локальной машине с Nvidia 3060ti mobile, а вторая на удаленной машине с Nvidia Tesla P100.

Посмотрим на скорость сходимости трансформера (TF) с параметром $ L = 12 $ и Looped TFs с параметрами $ L = 1; b = \{5, 10, 20, 25\} T= \{10, 20\}$. Были использованы следующие параметры Heads=4, dims=10, points=31. Потери масштабированы на размерность регрессии.

|                                                    |                                               |
|----------------------------------------------------|-----------------------------------------------|
| ![](../images/Compare_losses_b_n_t_10.png "Title") | ![](../images/Compare_losses_b_n.png "Title") |

Как можно видеть из графиков, Looped TFs при увеличении $ b $ дает улучшение метрик. При количестве параметров в 12 раз меньшим, чем у обычного трансформера, он показывает неплохие показатели. Конфигурации для запуска моделей взяты из папки эксперимента №5.

### Итеративные свойства
По скольку в статье подразумевается что Looped TFs имеют итеративные свойства, мы можем их проверить на примере линейной регрессии.
Для этого обучим Looped TFs со значениями $ b=\{5, 10, 20\}; T=\{ 20, 10\} $  и посмотрим на их сходимость при больших $ b $.

|           |       |
|-----------|-------|
| ![alt text](../images/check_for_convergence_properties.png "Title")    | ![alt text](../images/check_for_scheduling_convergence_properties_noisy_linear_regression_T_10_short.png "Title") |

Из графика можем видеть что при большем $ b $ на тренировке Looped TF показывает себя лучше при $ b > 5 $.  Блокнот `experiment_schedule.ipynb`. 


### Дополнительно
Появилась гипотеза о том, что шедулинг может помочь для обобщающих способностей модели. Увеличивая параметр $ b $ во время тренировки можно получить лучшую обобщающую способность, однако в процессе эксперимента со слабыми моделями это явно не подтвердилось. При тренировке с вариативным & b & и , модель дает меньшую ошибку на большем количестве итераций при $ T=20 $, однако при $ T=10 $ утверждать то же не можем.  Опыты проводились в блокноте `experiment_shedule.ipynb` и в блокноте `experiment_shedule_2.ipynb`.

|           |                                                                                              |
|-----------|----------------------------------------------------------------------------------------------|
| ![](../images/check_for_scheduling_convergence_properties.png "Title")    | ![](../images/check_for_scheduling_convergence_properties_train_steps_10000.png "Title")     |





Из графика можно видеть, что в отличии от моделей с параметрами $ b=5; b=10; b=15 $, у модели с варьированием $ b $ на этапе тренировки показатели точности сильно разнятся. 

### Попробуем брать только N последних токенов
Проверим гипотезу

> ***Гипотеза***
> 
> В таком случае, модель имеет возможность использовать часть токенов как хранилище информации с предыдущего шага.

Чтобы понять на сколько мы можем обрезать количество токенов которые подаются модель замаскируем часть из них и посмотрим на метрики.


На графиках ниже мы можем видеть вполне ожидаемый результат, для моделей с $ b = \{5, 10, 20\} $ точность ухудшается вместе с уменьшением количества токенов поступающих на вход. 

![](../images/check_last_n_tokens_quality.png "Title") 

Также интересно, что при удваивании количества итераций при инференсе мы получаем ошибку больше на каждом этапе отбрасывания $ n $ токенов. Это может происходить в результате того, что модели при увеличенном количестве итераций, от изначальной тренировки, более чувствительны к нехватке токенов. 

## Looped n-layers.


In [1]:
from models import TransformerModelLoopedLastNTokens
from train import get_task_sampler
transformer_model = TransformerModelLoopedLastNTokens(
    n_dims=2,
    n_positions=101,
    n = 2,
    n_embd=4,
    n_layer=1,
    n_head=2,
    pred_type="regression",
).cuda()

number of parameters: 0.00M


In [12]:
task_sampler = get_task_sampler(
    task_name="linear_regression",
    batch_size=1,
    n_points=10,
    n_dims=2,
    n_dims_truncated=3,
    device="cuda"
)

real_task = task_sampler()
xs, ys = real_task.xs.float(), real_task.ys.float()
n_loops = 3  # K
n_loop_window = 20

horizon_start = max(0, n_loops - n_loop_window)
## forward pass 
B, n, d_in = xs.shape
zs = transformer_model._combine(xs, ys)  # [B, n, d_in], [B, n], [B, n] -> [B, 2n, d_in + 1]

In [13]:
transformer_model(xs, ys, horizon_start, n_loops)

[tensor([[-0.3710, -0.3643, -0.7270, -0.1246, -0.0335, -0.1669, -0.0703, -0.5157,
          -0.7283, -0.5444]], device='cuda:0', grad_fn=<SelectBackward0>),
 tensor([[-0.3710, -0.3643, -0.7270, -0.1246, -0.0335, -0.1669, -0.0703, -0.5157,
          -0.7284, -0.5444]], device='cuda:0', grad_fn=<SelectBackward0>),
 tensor([[-0.3710, -0.3643, -0.7270, -0.1246, -0.0335, -0.1669, -0.0703, -0.5157,
          -0.7284, -0.5444]], device='cuda:0', grad_fn=<SelectBackward0>)]

In [14]:
zs

tensor([[[ 0.5870,  0.1149],
         [-0.2003,  0.0000],
         [-2.0130, -0.2876],
         [ 0.6235,  0.0000],
         [ 0.6402, -0.9374],
         [ 0.4157,  0.0000],
         [-0.6628,  0.1530],
         [ 0.0575,  0.0000],
         [ 0.7804,  1.3274],
         [-0.9673,  0.0000],
         [-1.4634,  2.8384],
         [-1.3653,  0.0000],
         [ 0.0658,  1.4392],
         [-0.8736,  0.0000],
         [ 0.6056, -0.1975],
         [-0.0181,  0.0000],
         [-0.1198, -0.3768],
         [ 0.2517,  0.0000],
         [-0.3114, -0.3883],
         [ 0.3016,  0.0000]]], device='cuda:0')

In [15]:
transformer_model._read_in(zs), transformer_model._read_in(zs).shape

(tensor([[[-9.2080e-02, -3.2012e-01, -3.4652e-01,  2.7293e-02],
          [-2.1660e-01, -1.8346e-01, -1.9326e-01, -8.6382e-02],
          [-4.9213e-01,  1.1829e-01,  1.5017e-01, -3.5361e-01],
          [-2.8036e-02, -3.9375e-01, -4.0285e-01,  3.7657e-03],
          [ 4.3008e-01, -9.2265e-01, -7.9086e-01, -2.1887e-01],
          [-7.5596e-02, -3.4071e-01, -3.4999e-01, -1.8972e-02],
          [-3.9658e-01,  2.0201e-02, -1.2983e-02, -1.0036e-01],
          [-1.5758e-01, -2.4927e-01, -2.5886e-01, -5.8169e-02],
          [-6.3543e-01,  3.0912e-01,  1.0066e-01,  3.3881e-01],
          [-3.9215e-01,  1.2325e-02,  1.8709e-03, -1.7031e-01],
          [-1.8812e+00,  1.7275e+00,  1.2900e+00,  4.5508e-01],
          [-4.8325e-01,  1.1392e-01,  1.0313e-01, -2.1386e-01],
          [-8.5315e-01,  5.5408e-01,  3.2821e-01,  2.8738e-01],
          [-3.7071e-01, -1.1593e-02, -2.1967e-02, -1.6006e-01],
          [ 6.3601e-02, -4.9973e-01, -4.7917e-01, -4.5484e-02],
          [-1.7488e-01, -2.2998e-01, -2.

In [16]:
x = transformer_model._read_in(zs)

In [19]:
import torch
n = 6
print(x.shape)
x_mask = torch.zeros((x.shape[0], x.shape[1] - n * 2, x.shape[2])).cuda()
print(x_mask.shape)
x_n = x[:, :n * 2, :]
print(x_n.shape)
torch.cat([x_n, x_mask], dim=1)


torch.Size([1, 20, 4])
torch.Size([1, 8, 4])
torch.Size([1, 12, 4])


tensor([[[-9.2080e-02, -3.2012e-01, -3.4652e-01,  2.7293e-02],
         [-2.1660e-01, -1.8346e-01, -1.9326e-01, -8.6382e-02],
         [-4.9213e-01,  1.1829e-01,  1.5017e-01, -3.5361e-01],
         [-2.8036e-02, -3.9375e-01, -4.0285e-01,  3.7657e-03],
         [ 4.3008e-01, -9.2265e-01, -7.9086e-01, -2.1887e-01],
         [-7.5596e-02, -3.4071e-01, -3.4999e-01, -1.8972e-02],
         [-3.9658e-01,  2.0201e-02, -1.2983e-02, -1.0036e-01],
         [-1.5758e-01, -2.4927e-01, -2.5886e-01, -5.8169e-02],
         [-6.3543e-01,  3.0912e-01,  1.0066e-01,  3.3881e-01],
         [-3.9215e-01,  1.2325e-02,  1.8709e-03, -1.7031e-01],
         [-1.8812e+00,  1.7275e+00,  1.2900e+00,  4.5508e-01],
         [-4.8325e-01,  1.1392e-01,  1.0313e-01, -2.1386e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.000